# Demo of PSyKI

This is a brief demonstration of PSyKI.
All main symbolic knowledge injection workflow steps will be addressed.


Some utility imports.

In [1]:
from psyki.logic.datalog.grammar.adapters.antlr4 import get_formula_from_string
from test.resources.rules.poker import FEATURE_MAPPING as POKER_FEATURE_MAPPING
from test.resources.rules.poker import CLASS_MAPPING as POKER_CLASS_MAPPING
from test.utils import get_processed_dataset
from test.resources.rules import get_rules

Loading poker dataset and separation into train and test set.

In [2]:
feature_mapping = POKER_FEATURE_MAPPING
class_mapping = POKER_CLASS_MAPPING
train_x, train_y, test_x, test_y = get_processed_dataset('poker')
train_x

array([[ 1., 10.,  1., ..., 12.,  1.,  1.],
       [ 2., 11.,  2., ..., 12.,  2.,  1.],
       [ 3., 12.,  3., ..., 10.,  3.,  1.],
       ...,
       [ 2.,  1.,  2., ...,  1.,  4., 13.],
       [ 2., 12.,  4., ..., 12.,  4.,  9.],
       [ 1.,  7.,  3., ...,  8.,  3.,  7.]])

Import Datalog poker rules and parsing into Formula data structure.

In [3]:
poker_rules = get_rules('poker')
formulae = [get_formula_from_string(rule) for rule in poker_rules]
str(formulae[0])

'pair(S1,R1,S2,R2,S3,R3,S4,R4,S5,R5)←((R1)=(R2))'

## Injection via structuring using NetworkComposer

In [4]:
from tensorflow.keras import Input
from psyki.ski.injectors import NetworkComposer
from tensorflow.keras.models import Model
from test.utils import get_mlp
from tensorflow.python.framework.random_seed import set_random_seed

set_random_seed(0)
input_features = Input((10,), name='Input')
network = get_mlp(input_layer=input_features, output=10, layers=3, neurons=128, activation_function='relu', last_activation_function='softmax')
model = Model(input_features, network)
injector = NetworkComposer(model, feature_mapping, layer=len(model.layers)-2)
predictor = injector.inject(formulae)
predictor.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
predictor.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 10)]         0           []                               
                                                                                                  
 lambda_416 (Lambda)            (None, 1)            0           ['Input[0][0]']                  
                                                                                                  
 lambda_417 (Lambda)            (None, 1)            0           ['Input[0][0]']                  
                                                                                                  
 lambda_418 (Lambda)            (None, 1)            0           ['Input[0][0]']                  
                                                                                            

### Training

In [5]:
predictor.fit(train_x, train_y, verbose=1, batch_size=32, epochs=20)

Epoch 1/20


2022-06-28 10:04:13.141070: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


782/782 [==============================] - 10s 4ms/step - loss: 0.8014 - accuracy: 0.7303
Epoch 2/20
782/782 [==============================] - 4s 4ms/step - loss: 0.3914 - accuracy: 0.9293
Epoch 3/20
782/782 [==============================] - 3s 4ms/step - loss: 0.2272 - accuracy: 0.9545
Epoch 4/20
782/782 [==============================] - 3s 4ms/step - loss: 0.1434 - accuracy: 0.9805
Epoch 5/20
782/782 [==============================] - 3s 4ms/step - loss: 0.0946 - accuracy: 0.9910
Epoch 6/20
782/782 [==============================] - 3s 4ms/step - loss: 0.0647 - accuracy: 0.9963
Epoch 7/20
782/782 [==============================] - 3s 4ms/step - loss: 0.0451 - accuracy: 0.9985
Epoch 8/20
782/782 [==============================] - 3s 4ms/step - loss: 0.0323 - accuracy: 0.9994
Epoch 9/20
782/782 [==============================] - 3s 4ms/step - loss: 0.0233 - accuracy: 0.9995
Epoch 10/20
782/782 [==============================] - 3s 4ms/step - loss: 0.0172 - accuracy: 0.9996
Epoch 11/

### Evaluation
Loss and accuracy

In [6]:
loss, accuracy = predictor.evaluate(test_x, test_y)
print('Loss: ' + str(loss))
print('Accuracy: ' + str(accuracy))

31250/31250 [==============================] - 129s 4ms/step - loss: 0.0027 - accuracy: 0.9997
Loss: 0.0026770946569740772
Accuracy: 0.9997010231018066


In [7]:
from test.utils import get_class_accuracy

accuracies, weights = get_class_accuracy(predictor, test_x, test_y)
for label, index in sorted(class_mapping.items(), key=lambda i: i[1]):
    print('Accuracy of class ' + label + ': ' + str(accuracies[index]))

Accuracy of class nothing: 1.0
Accuracy of class pair: 1.0
Accuracy of class two: 1.0
Accuracy of class three: 1.0
Accuracy of class straight: 1.0
Accuracy of class flush: 0.998496993987976
Accuracy of class full: 0.9396067415730337
Accuracy of class four: 0.11739130434782609
Accuracy of class straight_flush: 0.5833333333333334
Accuracy of class royal_flush: 0.3333333333333333


Macro F1-measure

In [8]:
from test.utils import get_f1

f1 = get_f1(predictor, test_x, test_y)
print('macro F1-measure: ' + str(f1))

macro F1-measure: 0.8210408308532431


## Injection via constraining using LambdaLayer

In [9]:
from psyki.ski.injectors import LambdaLayer

str(formulae[0])
set_random_seed(0)
input_features = Input((10,), name='Input')
network = get_mlp(input_layer=input_features, output=10, layers=3, neurons=128, activation_function='relu', last_activation_function='softmax')
model = Model(input_features, network)
injector = LambdaLayer(model, class_mapping, feature_mapping, 1.5)
predictor = injector.inject(formulae)
predictor.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
predictor.summary()

Model: "constrained_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 10)]         0           []                               
                                                                                                  
 dense_619 (Dense)              (None, 128)          1408        ['Input[0][0]']                  
                                                                                                  
 dense_620 (Dense)              (None, 128)          16512       ['dense_619[0][0]']              
                                                                                                  
 dense_621 (Dense)              (None, 10)           1290        ['dense_620[0][0]']              
                                                                                  

### Training

In [10]:
predictor.fit(train_x, train_y, verbose=1, batch_size=32, epochs=100)

Epoch 1/100
782/782 [==============================] - 15s 6ms/step - loss: 1.4392 - accuracy: 0.2200
Epoch 2/100
782/782 [==============================] - 5s 7ms/step - loss: 1.3728 - accuracy: 0.2930
Epoch 3/100
782/782 [==============================] - 5s 6ms/step - loss: 1.3531 - accuracy: 0.3260
Epoch 4/100
782/782 [==============================] - 5s 6ms/step - loss: 1.3373 - accuracy: 0.3530
Epoch 5/100
782/782 [==============================] - 5s 6ms/step - loss: 1.3259 - accuracy: 0.3671
Epoch 6/100
782/782 [==============================] - 5s 6ms/step - loss: 1.3131 - accuracy: 0.3766
Epoch 7/100
782/782 [==============================] - 5s 6ms/step - loss: 1.3022 - accuracy: 0.3820
Epoch 8/100
782/782 [==============================] - 5s 6ms/step - loss: 1.2931 - accuracy: 0.3842
Epoch 9/100
782/782 [==============================] - 5s 6ms/step - loss: 1.2778 - accuracy: 0.3883
Epoch 10/100
782/782 [==============================] - 5s 6ms/step - loss: 1.2635 - accur

### Remove constraints and evaluation
Loss and Accuracy

In [11]:
predictor = predictor.remove_constraints()
predictor.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
loss, accuracy = predictor.evaluate(test_x, test_y)
print('Loss: ' + str(loss))
print('Accuracy: ' + str(accuracy))

31250/31250 [==============================] - 9s 289us/step - loss: 0.1893 - accuracy: 0.9497
Loss: 0.18925563991069794
Accuracy: 0.9496650099754333


In [12]:
accuracies, weights = get_class_accuracy(predictor, test_x, test_y)
for label, index in sorted(class_mapping.items(), key=lambda i: i[1]):
    print('Accuracy of class ' + label + ': ' + str(accuracies[index]))

Accuracy of class nothing: 0.9943356962863795
Accuracy of class pair: 0.9682412697811588
Accuracy of class two: 0.5237705262273739
Accuracy of class three: 0.7144074617679087
Accuracy of class straight: 0.44684684684684683
Accuracy of class flush: 0.053607214428857713
Accuracy of class full: 0.22191011235955055
Accuracy of class four: 0.10434782608695652
Accuracy of class straight_flush: 0.0
Accuracy of class royal_flush: 0.0


Macro F1-measure

In [13]:
f1 = get_f1(predictor, test_x, test_y)
print('macro F1-measure: ' + str(f1))

macro F1-measure: 0.43701232262534695


## The same network without knowledge injection

In [14]:
set_random_seed(0)
input_features = Input((10,), name='Input')
network = get_mlp(input_layer=input_features, output=10, layers=3, neurons=128, activation_function='relu', last_activation_function='softmax')
model = Model(input_features, network)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 10)]              0         
                                                                 
 dense_622 (Dense)           (None, 128)               1408      
                                                                 
 dense_623 (Dense)           (None, 128)               16512     
                                                                 
 dense_624 (Dense)           (None, 10)                1290      
                                                                 
Total params: 19,210
Trainable params: 19,210
Non-trainable params: 0
_________________________________________________________________


### Training

In [15]:
model.fit(train_x, train_y, verbose=1, batch_size=32, epochs=100)

Epoch 1/100
782/782 [==============================] - 0s 474us/step - loss: 1.0044 - accuracy: 0.5090
Epoch 2/100
782/782 [==============================] - 0s 477us/step - loss: 0.9620 - accuracy: 0.5361
Epoch 3/100
782/782 [==============================] - 0s 471us/step - loss: 0.9504 - accuracy: 0.5483
Epoch 4/100
782/782 [==============================] - 0s 470us/step - loss: 0.9433 - accuracy: 0.5529
Epoch 5/100
782/782 [==============================] - 0s 470us/step - loss: 0.9348 - accuracy: 0.5577
Epoch 6/100
782/782 [==============================] - 0s 477us/step - loss: 0.9270 - accuracy: 0.5672
Epoch 7/100
782/782 [==============================] - 0s 459us/step - loss: 0.9189 - accuracy: 0.5698
Epoch 8/100
782/782 [==============================] - 0s 469us/step - loss: 0.9106 - accuracy: 0.5749
Epoch 9/100
782/782 [==============================] - 0s 494us/step - loss: 0.8998 - accuracy: 0.5831
Epoch 10/100
782/782 [==============================] - 0s 476us/step - l

### Evaluation
Loss and Accuracy

In [16]:
loss, accuracy = model.evaluate(test_x, test_y)
print('Loss: ' + str(loss))
print('Accuracy: ' + str(accuracy))

31250/31250 [==============================] - 9s 292us/step - loss: 0.0958 - accuracy: 0.9834
Loss: 0.09576362371444702
Accuracy: 0.983398973941803


In [17]:
accuracies, weights = get_class_accuracy(model, test_x, test_y)
for label, index in sorted(class_mapping.items(), key=lambda i: i[1]):
    print('Accuracy of class ' + label + ': ' +str(accuracies[index]))

Accuracy of class nothing: 0.9961014267501183
Accuracy of class pair: 0.9979005817779019
Accuracy of class two: 0.8837092100289782
Accuracy of class three: 0.8831494720893897
Accuracy of class straight: 0.33153153153153153
Accuracy of class flush: 0.053607214428857713
Accuracy of class full: 0.27879213483146065
Accuracy of class four: 0.017391304347826087
Accuracy of class straight_flush: 0.0
Accuracy of class royal_flush: 0.0


Macro F1-measure

In [18]:
f1 = get_f1(model, test_x, test_y)
print('macro F1-measure: ' + str(f1))

macro F1-measure: 0.4629104593757497
